In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

/Users/juliasbardelatti/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
raca_0_ate_5_anos = pd.read_csv("../0_ate_5/bolsa_familia/bolsa_familia.csv")
raca_0_ate_5_anos['faixa_etaria'] = '0 a 5 anos'

raca_5_ate_10_anos = pd.read_csv("../5_ate_10/bolsa_familia/bolsa_familia.csv")
raca_5_ate_10_anos['faixa_etaria'] = '5 a 10 anos'

raca_adolescente = pd.read_csv("../adolescente/bolsa_familia/bolsa_familia.csv")
raca_adolescente['faixa_etaria'] = 'Adolescentes (10 a 18 anos)'

df_completo = pd.concat([raca_0_ate_5_anos, raca_5_ate_10_anos, raca_adolescente])


In [4]:
df_completo['total'] = df_completo['total'].astype(str).str.replace('.', '', regex=True)
df_completo['total'] = pd.to_numeric(df_completo['total'], errors='coerce')

In [5]:
colunas_para_converter = ['qtd_magreza_acentuada', 'qtd_magreza', 'qtd_eutrofia', 'qtd_risco_sobrepeso', 'qtd_sobrepeso', 'qtd_obesidade'] 

for coluna in colunas_para_converter:
    df_completo[coluna] = df_completo[coluna].astype(str).str.strip()  
    df_completo[coluna] = df_completo[coluna].str.replace(r'\.', '', regex=True)  
    df_completo[coluna] = pd.to_numeric(df_completo[coluna], errors='coerce') 

In [6]:
colunas_para_somar = ['qtd_magreza_acentuada', 'qtd_magreza', 'qtd_eutrofia', 'qtd_risco_sobrepeso', 'qtd_sobrepeso', 'qtd_obesidade'] 
df_completo = df_completo[df_completo['regiao'].str.startswith(('TOTAL REGIÃO', 'TOTAL BRASIL'), na=False)]
df_grouped = df_completo.groupby(['regiao', 'ano'])[colunas_para_somar].sum().reset_index()
df_grouped['regiao'].unique()

array(['TOTAL BRASIL', 'TOTAL REGIÃO CENTRO-OESTE',
       'TOTAL REGIÃO NORDESTE', 'TOTAL REGIÃO NORTE',
       'TOTAL REGIÃO SUDESTE', 'TOTAL REGIÃO SUL'], dtype=object)

In [7]:

df_grouped['total'] = df_grouped[colunas_para_somar].sum(axis=1)
for coluna in colunas_para_somar:
    df_grouped[f'{coluna}_perc'] = (df_grouped[coluna] / df_grouped['total']) 


In [8]:
df_grouped['perc_total_obesidade'] = df_grouped['qtd_sobrepeso_perc'] + df_grouped['qtd_obesidade_perc']

In [9]:
df_grouped = df_grouped[['regiao', 'perc_total_obesidade', 'ano']]

mapeamento = {
    'TOTAL REGIÃO CENTRO-OESTE': 'Centro-oeste',
    'TOTAL REGIÃO NORDESTE': 'Nordeste',
    'TOTAL REGIÃO NORTE': 'Norte',
    'TOTAL REGIÃO SUDESTE': 'Sudeste',
    'TOTAL REGIÃO SUL': 'Sul',
    'TOTAL BRASIL': 'Total Brasil'
}

df_grouped['regiao'] = df_grouped['regiao'].replace(mapeamento)